# Python Selenium untuk Scraping Rating Data Traveloka
hotel_link_list diambil dari data txtnya alfi

In [ ]:
# Download Selenium
!pip install selenium
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!gdown --id 1bDx9hdqPsz8zxf7Pmj3HCSdIxj8OPvQT


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 5.5 MB/s 
     |████████████████████████████████| 138 kB 70.5 MB/s 
     |████████████████████████████████| 359 kB 71.4 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 4.0 MB 52.0 MB/s 
     |████████████████████████████████| 58 kB 3.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in inde

In [ ]:
# get listOfLink from links.txt
with open('/content/links.txt') as f:
    string = f.readline()
    listOfLink = string[1:-1].replace("'", "").split(", ")

In [ ]:
import sys
import time
import re
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
def getHotelIdFromLink(link):
  result = re.search(r"[a-zA-Z\/:.-]*([0-9]*)?", link)
  return result.group(1)

In [ ]:
def checkEmptyString(listOfString):
  for string in listOfString:
    if string == '':
      return False
  return True

In [ ]:
def createUserItem(name, rating, review, travel_purpose, user_item): 
  user_item['name'] = name
  user_item['rating'] = rating
  user_item['review'] = review
  user_item['travel_purpose'] = travel_purpose
  if not checkEmptyString([name, rating, review, travel_purpose]):
    return False
  return user_item

In [ ]:
def getBlockReview(wd, hotel_id, db):
  review_container = wd.find_element_by_xpath('//div[contains(@class, "_1xx34")]')
  try :
    load_more_button = wd.find_element_by_xpath('//div[contains(@class, "_1hiwh")]')
  except:
      return;
  
  x_path_name = ".//div[@class='css-901oao r-1sixt3s r-1b43r93 r-b88u0q r-rjixqe r-fdjqy7']"
  x_path_rating = ".//div[@class='css-901oao r-1i6uqv8 r-1sixt3s r-1b43r93 r-majxgm r-rjixqe r-fdjqy7']"
  x_path_review = ".//div[@class='css-901oao r-1sixt3s r-1b43r93 r-majxgm r-rjixqe r-fdjqy7']"
  x_path_travel_purpose = ".//div[@class='css-901oao r-1wb8tzg r-1sixt3s r-1enofrn r-majxgm r-135wba7 r-fdjqy7']"

  total_review = 0
  temp_idx = 0

  while total_review < 14:
    review_list = review_container.find_elements_by_xpath('//div[contains(@class, "_1wzEb")]/div')
    for i in range(temp_idx, temp_idx+10):
      try:
        id = review_list[i].get_attribute('id')
        name = review_list[i].find_element_by_xpath(x_path_name).text
        rating = review_list[i].find_element_by_xpath(x_path_rating).text
        review = review_list[i].find_element_by_xpath(x_path_review).text
        try:
          travel_purpose = review_list[i].find_element_by_xpath(x_path_travel_purpose).text
        except:
          travel_purpose = ''
        user_item = {
            'user_id': id,
            'hotel_id': hotel_id
        }
        user_item = createUserItem(name, rating, review, travel_purpose, user_item)
        if user_item :
          db.append(user_item)
          total_review += 1
          review_list[i+1]
      except:
        total_review = 15

    wd.execute_script("arguments[0].click();", load_more_button)
    time.sleep(8)
    temp_idx = i + 1

In [ ]:
def getReviewHotel(wd, current_url, db):
  wd.get(current_url)
  hotel_id = getHotelIdFromLink(current_url)

  # pilih rendah ke tinggi
  rendah_ke_tinggi_button = wd.find_element_by_xpath("//div[@class='_2oYpR']/ul/li[2]")
  wd.execute_script("arguments[0].click();", rendah_ke_tinggi_button)
  time.sleep(8)

  getBlockReview(wd, hotel_id, db)

  tinggi_ke_rendah_button = wd.find_element_by_xpath("//div[@class='_2oYpR']/ul/li[3]")
  wd.execute_script("arguments[0].click();", tinggi_ke_rendah_button)
  time.sleep(8)

  getBlockReview(wd, hotel_id, db)

In [ ]:
import csv
def export_csv(filename, column, data):
    """Export list of dictionary to CSV"""
    try:
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=column)
            writer.writeheader()
            writer.writerows(data)
    except IOError:
        print("I/O error")

In [ ]:
def getHotelPhoto(wd, current_url, db):
  hotel_photo = {}
  wd.get(current_url)
  hotel_id = getHotelIdFromLink(current_url)
  try:
    x_path_image = "//img[@class='gWtZU']"
    image = wd.find_element_by_xpath(x_path_image)
    hotel_photo = {
        'hotel_id': hotel_id,
        'link': image.get_attribute('src')
    }
  except:
      hotel_photo = {
       'hotel_id': hotel_id,
       'link': ""   
      }
  print(hotel_photo)
  db.append(hotel_photo)

In [ ]:
db = []
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
for link in listOfLink:
  dbTemp = []
  getReviewHotel(wd, link, dbTemp)
  db += dbTemp
  print(link, dbTemp)

print()
print("semua", db)

In [ ]:
column = ['user_id', 'hotel_id', 'name', 'rating', 'review', 'travel_purpose']
export_csv('/content/review.csv', column, db)

In [ ]:
Hotel Image
db = []

In [ ]:
# isi manual
current_url = listOfLink[71]
print(current_url)
print(getHotelIdFromLink(current_url))

https://m.traveloka.com/en-id/hotel/indonesia/merusaka-nusa-dua--3000010000329?contexts=%7B%22sourceHotelDetail%22%3A%22MWEB.1732895188531044127%22%2C%22inventoryRateKey%22%3A%22povEwB3ZzsU2C6pd%2B6MdUxPQfH0zjQrhnrQN8muKP1lM0tKsNUio4Rxug1zOzHJilcvZa8YdqqIySdppe7dolTIewZlxfjiOGz6oXLTucMvN0%2BgQ3yZPvZcoHwUFNN%2BZpcr%2F%2FNa8o9ElfN8zOYhzzZeg0X586laFpeUViPi9rxAm17MfVVF2hBczGx0prvFIIvEWeildVo7nbHxIWcXPVNGysXwP9P%2F6jQTjnyKeTv%2ByqYmxSenRKFEfSHS0XO3ksOocFaHL1Wb9vQXntcm%2BDw%3D%3D%22%7D&amp;spec=31-12-2022.1-1-2023.1.1.HOTEL.3000010000329.MERUSAKA%20Nusa%20Dua.1&amp;loginPromo=1
3000010000329


In [ ]:
dbTemp = []
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
print(len(listOfLink))
count = 79
for link in listOfLink[80:247]:
  count += 1
  getHotelPhoto(wd, link, dbTemp)
  print(count, link)

print("semua", dbTemp)
db += dbTemp
print(len(db), db)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


247


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  import sys


{'hotel_id': '65538', 'link': 'https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/10001457-40fa3a4085a4938316128d613a7df561.jpeg?tr=q-40,c-at_max,w-740,h-500&_src=imagekit'}
80 https://m.traveloka.com/en-id/hotel/indonesia/the-patra-bali-resort--villas-65538?contexts=%7B%22sourceHotelDetail%22%3A%22MWEB.1732895216984152370%22%2C%22inventoryRateKey%22%3A%22UBh6NEbrm3oRvPz965TEQYx6yJl1TnvT4YG%2Byjorw5RM0tKsNUio4Rxug1zOzHJilcvZa8YdqqIySdppe7dolTIewZlxfjiOGz6oXLTucMvN0%2BgQ3yZPvZcoHwUFNN%2BZpcr%2F%2FNa8o9ElfN8zOYhzzZeg0X586laFpeUViPi9rxD11szoNIjqyEaaBF0%2FzcYlIvEWeildVo7nbHxIWcXPVNGysXwP9P%2F6jQTjnyKeTv%2ByqYmxSenRKFEfSHS0XO3ksOocFaHL1Wb9vQXntcm%2BDw%3D%3D%22%7D&amp;spec=31-12-2022.1-1-2023.1.1.HOTEL.65538.The%20Patra%20Bali%20Resort%20%26%20Villas.1&amp;loginPromo=1
{'hotel_id': '400056', 'link': 'https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/10021723-2560x1

In [ ]:
print(len(db), db)

247 [{'hotel_id': '3000010039024', 'link': 'https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/10039024-3182x2122-FIT_AND_TRIM-593d8b6835e9ead9a7e8c6e9a2e7507b.jpeg?tr=q-40,c-at_max,w-740,h-500&_src=imagekit'}, {'hotel_id': '1000000509788', 'link': 'https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/10007041-78a99192f82684bf70f7e21a9df8ab63.jpeg?tr=q-40,c-at_max,w-740,h-500&_src=imagekit'}, {'hotel_id': '409746', 'link': 'https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/10002860-868x1024-FIT_AND_TRIM-f0f9306c6aed589fe36d2b2e3d654bb9.jpeg?tr=q-40,c-at_max,w-740,h-500&_src=imagekit'}, {'hotel_id': '100215', 'link': 'https://ik.imagekit.io/tvlk/apr-asset/dgXfoyh24ryQLRcGq00cIdKHRmotrWLNlvG-TxlcLxGkiDwaUSggleJNPRgIHCX6/hotel/asset/10001785-1690x1148-FIT_AND_TRIM-b480d344449ed28dca5241b57a44052e.jpeg

In [ ]:
column = ['hotel_id', 'link']
export_csv('/content/photoHotel.csv', column, db)